In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
folder_path = "../study"
stats_path = folder_path + "/stats"

In [3]:
from myTransformers import FilterSmall
from sklearn.model_selection import train_test_split

# Reading the whole dataset
df = pd.read_csv("%s/dataset.csv" % stats_path, index_col="ID")
print("Before:", df.shape)

# Removing the NaN values
df = df.dropna(axis=0, how="any")

# Separing Data from labels and removing benzo and type
X = df.drop(["resp", "respPart"], axis=1)
y = df["resp"]

# splitting training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, shuffle=True, stratify=y)
print("X_train", X_train.shape)
print("X_test:", X_test.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)
print(y_test)

# Removing too small dMRI features
tooSmall = X_train.columns[np.array(np.min(np.abs(X), axis=0) <= 1e-12)]
X_train = X_train.drop(tooSmall, axis=1)
X_test = X_test.drop(tooSmall, axis=1)

col_dMRI = X_train.filter(regex=r'mean|std|skew|kurt').columns
col_nTract = X_train.filter(regex=r'nTracts').columns
col_cont = ["age", "therapy_duration", "epilepsy_onset_age", "epilepsy_duration", *col_nTract, *col_dMRI]
col_disc = ["sex", "AEDs", "benzo", "epilepsy_type"]
print("X_train After:", X_train.shape)

ModuleNotFoundError: No module named 'sklearn.utils._param_validation'

In [5]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, mutual_info_classif
from sklearn.linear_model import LogisticRegression, ElasticNet, LinearRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, roc_auc_score

def myScoreFunc(X, y):
    mi_score = mutual_info_classif(X, y)
    f_score = f_classif(X, y)[0]
    return mi_score + f_score

models = {
    # SVM
    "linearSVM" : (
        LinearSVC(), 
        {
            "selection__k" : (2, 3, 5, 7, 9, 11, 13, 17, 19),
            "classifier__C" : (1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1),
        }
    ),
    # "SVM" : (
    #     SVC(),
    #     [
    #         {
    #             "selection__k" : (2, 3, 5, 7, 9, 11, 13, 17, 19),
    #             "classifier__C" :(1e-3, 1e-2, 1e-1, 1),
    #             "classifier__kernel" : ['linear'], 
    #         },
    #         {
    #             "selection__k" : (2, 3, 5, 7, 9, 11, 13, 17, 19),
    #             "classifier__C" :(1e-3, 1e-2, 1e-1, 1),
    #             "classifier__kernel" : ['poly'],
    #             "classifier__degree" : (2, 3, 4),
    #         },
    #         {
    #             "selection__k" : (2, 3, 5, 7, 9, 11, 13, 17, 19),
    #             "classifier__C" :(1e-3, 1e-2, 1e-1, 1),
    #             "classifier__kernel" : ['rbf'], 
    #             "classifier__gamma" : (1e-3, 1e-2, 1e-1, 1),
    #         },
    #     ]
    # ),
    # # Linear 
    # "LogReg" : (
    #     LogisticRegression(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13, 17, 19),
    #         "classifier__penalty" : ["l2"],
    #         "classifier__dual" : [True],
    #         "classifier__C" :(1e-3, 1e-2, 1e-1),
    #     },
    # ),
    # "LinReg" : (
    #     LinearRegression(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13, 17, 19),
    #     }
    # ),
    # "ridgeReg" : (
    #     RidgeClassifier(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #         "classifier__alpha" : (0.5, 1, 5, 10, 20, 40)
    #     }
    # ),
    # "elasticNet" : (
    #     ElasticNet(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #         "classifier__alpha" : (0.5, 1, 5, 10, 20, 40),  # 0 == Linear Regression
    #         "classifier__l1_ratio" : (0.5, 1), # 0 == Ridge Regression, 1 == Lasso Regression
    #     },
    # ),
    # # Nearest Neighbors
    # "neighbors" : (
    #     KNeighborsClassifier(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #         "classifier__n_neighbors" : (2, 3, 5),
    #         "classifier__weights" : ("uniform", "distance"),
    #     }
    # ),
    # # Naive Bayes
    # "gaussianNaive" : (
    #     GaussianNB(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #     }
    # ),
    # "multinomialNaive" : (
    #     MultinomialNB(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #     }
    # ),
    # "complementNaive" : (
    #     ComplementNB(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #     }
    # ),
    # "bernulliNaive" : (
    #     BernoulliNB(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #     }
    # ),
    # "categoricalNaive" : (
    #     CategoricalNB(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #     }
    # ),
    # # Tree
    # "tree" : (
    #     DecisionTreeClassifier(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #         "classifier__criterion" : ("gini", "entropy", "log_loss")
    #         # ccp_apha is a Regularization therm (to try)
    #     }
    # ),
    # # Ensemble
    # "forest" : (
    #     RandomForestClassifier(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #         "classifier__n_estimators" : (100, 500, 1000),
    #         # Check if make sense choose also the criterion (see the tree)
    #         "classifier__bootstrap" : [True],
    #         "classifier__max_samples" : [0.5],
    #         "classifier__max_features" : ["log2"],
    #         "classifier__warm_star" : [True],
    #         "classifier__oob_score" : [True],
    #         "classifier__max_depth" : [10, 20, 30],
    #         # ccp_alpha da checkare
    #     }
    # ),
    # "extraForest" : (
    #     ExtraTreesClassifier(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #         "classifier__n_estimators" : (100, 500, 1000),
    #         # Check if make sense choose also the criterion (see the tree)
    #         "classifier__bootstrap" : [True],
    #         "classifier__max_samples" : [0.5],
    #         "classifier__max_features" : ["log2"],
    #         "classifier__warm_star" : [True],
    #         "classifier__oob_score" : [True],
    #         "classifier__max_depth" : [10, 20, 30],
    #         # ccp_alpha da checkare
    #     }
    # ),
    # "gradientBoosting": (
    #     GradientBoostingClassifier(),
    #     {
    #         "selection__k" : (2, 3, 5, 7, 9, 11, 13),
    #         "classifier__n_estimators" : (100, 200, 500, 1000),
    #         "classifier__learning_rate" : (0.05, 0.1),
    #         # Check if make sense choose also the criterion (see the tree)
    #         "classifier__max_samples" : [0.5],
    #         "classifier__max_features" : ["log2"],
    #         "classifier__warm_star" : [True],
    #         "classifier__max_depth" : [10, 20, 30],
    #         "classifier__validation_fraction" : [0.20],
    #         "classifier__n_iter_no_change" : [50],
    #         # ccp_alpha da checkare
    #     }
    # )
}

sss = StratifiedShuffleSplit(n_splits=20, test_size=1/3)

fitted_models = {}

for modelName, (classifier, grid) in models.items():
    for scaler in [StandardScaler(), RobustScaler()]:
        for filter in [f_classif]:
            print("modelName:", modelName, ", scaler:", scaler.__class__.__name__, ", filter:", filter.__name__)

            # Scaling
            pre = ColumnTransformer(
                    [("scaling", scaler, col_cont)],
                    remainder="passthrough", # one hot or other stuff
                    n_jobs=-1
                )
            
            # Remove costant values
            # Yes, it's possible, since we have few data after the splitting and the cross validation is possible to have some features with same values. To remove them from the modeling we use the 
            varThres = VarianceThreshold()

            # Feature Selection
            selection = SelectKBest(
                    score_func=filter,
                )

            # Pipe
            pipe = Pipeline([
                ("pre", pre),
                ("varThres", varThres),
                ("selection", selection),
                ("classifier", classifier)
            ])

            model = GridSearchCV(
                estimator=pipe,
                param_grid=grid,
                scoring="roc_auc",
                n_jobs=-1,
                cv=sss,
            )

            model.fit(X_train, y_train)
            fitted_models[modelName] = model

            # Train
            print(modelName, "train score:", model.best_score_)
            print(model.best_params_) 

            idx = [param.__str__() for param in model.cv_results_["params"]]
            grid = pd.DataFrame(pd.concat([pd.DataFrame(model.cv_results_["mean_test_score"], index=idx, columns=["mean_test_score"]), pd.DataFrame(model.cv_results_["std_test_score"], index=idx, columns=["std_test_score"]), pd.DataFrame(model.cv_results_["rank_test_score"], index=idx, columns=["rank_test_score"])], axis=1))
            print(grid.sort_values("rank_test_score")[:10])

            # Test
            y_pred = model.predict(X_test)
            confusionMatrix = confusion_matrix(y_test, y_pred)  
            print(confusionMatrix)  
            print("Balanced Accuracy Score", balanced_accuracy_score(y_test, y_pred, adjusted=True))
            print("Area Under ROC", roc_auc_score(y_test, y_pred,))


modelName: linearSVM , scaler: StandardScaler , filter: f_classif
linearSVM train score: 0.7666666666666666
{'classifier__C': 1e-06, 'selection__k': 3}
[[0 2]
 [0 4]]
Balanced Accuracy Score 0.0
Area Under ROC 0.5
modelName: linearSVM , scaler: RobustScaler , filter: f_classif
linearSVM train score: 0.7666666666666666
{'classifier__C': 1e-06, 'selection__k': 3}
[[0 2]
 [0 4]]
Balanced Accuracy Score 0.0
Area Under ROC 0.5


In [54]:

idx = [param.__str__() for param in model.cv_results_["params"]]
grid = pd.DataFrame(pd.concat([pd.DataFrame(model.cv_results_["mean_test_score"], index=idx, columns=["mean_test_score"]), pd.DataFrame(model.cv_results_["std_test_score"], index=idx, columns=["std_test_score"]), pd.DataFrame(model.cv_results_["rank_test_score"], index=idx, columns=["rank_test_score"])], axis=1))
print(grid.sort_values("rank_test_score")[:10])

                                              mean_test_score  std_test_score   
{'classifier__C': 0.001, 'selection__k': 3}          0.766667        0.334996  \
{'classifier__C': 1e-06, 'selection__k': 3}          0.766667        0.334996   
{'classifier__C': 0.01, 'selection__k': 3}           0.766667        0.334996   
{'classifier__C': 0.0001, 'selection__k': 3}         0.766667        0.334996   
{'classifier__C': 1e-05, 'selection__k': 3}          0.766667        0.334996   
{'classifier__C': 0.1, 'selection__k': 3}            0.766667        0.334996   
{'classifier__C': 1, 'selection__k': 5}              0.750000        0.314024   
{'classifier__C': 0.001, 'selection__k': 5}          0.733333        0.326599   
{'classifier__C': 0.01, 'selection__k': 2}           0.733333        0.359011   
{'classifier__C': 0.01, 'selection__k': 5}           0.733333        0.326599   

                                              rank_test_score  
{'classifier__C': 0.001, 'selection__k': 3} 

In [37]:
[param.__str__() for param in model.cv_results_["params"]]

["{'classifier__C': 1e-06, 'selection__k': 2}",
 "{'classifier__C': 1e-06, 'selection__k': 3}",
 "{'classifier__C': 1e-06, 'selection__k': 5}",
 "{'classifier__C': 1e-06, 'selection__k': 7}",
 "{'classifier__C': 1e-06, 'selection__k': 9}",
 "{'classifier__C': 1e-06, 'selection__k': 11}",
 "{'classifier__C': 1e-06, 'selection__k': 13}",
 "{'classifier__C': 1e-06, 'selection__k': 17}",
 "{'classifier__C': 1e-06, 'selection__k': 19}",
 "{'classifier__C': 1e-05, 'selection__k': 2}",
 "{'classifier__C': 1e-05, 'selection__k': 3}",
 "{'classifier__C': 1e-05, 'selection__k': 5}",
 "{'classifier__C': 1e-05, 'selection__k': 7}",
 "{'classifier__C': 1e-05, 'selection__k': 9}",
 "{'classifier__C': 1e-05, 'selection__k': 11}",
 "{'classifier__C': 1e-05, 'selection__k': 13}",
 "{'classifier__C': 1e-05, 'selection__k': 17}",
 "{'classifier__C': 1e-05, 'selection__k': 19}",
 "{'classifier__C': 0.0001, 'selection__k': 2}",
 "{'classifier__C': 0.0001, 'selection__k': 3}",
 "{'classifier__C': 0.0001, 's